In [ ]:
!pip install requests

In [ ]:
import os
import requests
import csv
import re

PEXELS_API_KEY = "your actual key"  # Replace this with your actual key
headers = {
    "Authorization": PEXELS_API_KEY
}

# Sanitize filenames to avoid illegal characters
def clean_filename(text):
    return re.sub(r'[\\/*?:"<>|\n\r]', "_", text.replace(" ", "_")).strip()

def download_images(query, folder_name, total=100):
    os.makedirs(folder_name, exist_ok=True)
    image_data = []
    page = 1
    per_page = 30
    count = 0

    while count < total:
        url = f"https://api.pexels.com/v1/search?query={query}&per_page={per_page}&page={page}"
        try:
            res = requests.get(url, headers=headers)
            res.raise_for_status()
        except Exception as e:
            print(f"API error on page {page}: {e}")
            break

        photos = res.json().get("photos", [])
        if not photos:
            break  # No more results

        for photo in photos:
            if count >= total:
                break

            image_url = photo["src"]["original"]
            raw_alt = photo.get("alt", "unknown_location")
            alt = clean_filename(raw_alt)
            image_id = photo["id"]
            file_name = f"{alt}_{image_id}.jpg"
            file_path = os.path.join(folder_name, file_name)

            # Save image safely
            try:
                img = requests.get(image_url).content
                with open(file_path, "wb") as f:
                    f.write(img)

                # Save metadata
                image_data.append({
                    "filename": file_name,
                    "location": raw_alt
                })
                count += 1
            except Exception as e:
                print(f"Skipping {file_name} due to error: {e}")
                continue
        page += 1

    # Write metadata CSV
    csv_path = os.path.join(folder_name, "metadata.csv")
    with open(csv_path, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["filename", "location"])
        writer.writeheader()
        for data in image_data:
            writer.writerow(data)

    print(f" Done: {count} images saved in '{folder_name}'.")

#  Re-run for both places
download_images("Chennai nature beach greenery", "Chennai", 100)


✅ Done: 100 images saved in 'Chennai'.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/Kerala /content/drive/MyDrive/

In [ ]:
!cp -r /content/Chennai /content/drive/MyDrive/